<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/40_3_%3D%3D_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Дообучи русскоязычную модель GPT для генерации фейковых новостей.
2. Датасет необходимо найти самостоятельно.
3. Приведи 5-10 примеров сгенерированных фейковых новостей на русском языке.
4. Подбери параметры генерации, улучшите качество новостей.
5. Изучи model.generate более детально.
6. Добейся такой генерации, чтобы не было обрыва мысли, новость заканчивалась либо точкой, либо восклицательным знаком, либо вопросительным (если уместно).



In [6]:
# Устанавливаем необходимые пакеты
%%capture
!pip install kaggle

In [1]:
# Импортируем необходимые библиотеки
import kagglehub
import pandas as pd
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [2]:
# Определяем GPT модель и токенизатор
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [7]:
path = kagglehub.dataset_download("vorvit/news-lenta-20-23")
print("Path to dataset files:", path)
df = pd.read_csv(path + "/Lenta_sample.csv")
df.head(1)

Path to dataset files: /root/.cache/kagglehub/datasets/vorvit/news-lenta-20-23/versions/1


In [18]:
df.columns

Index(['docid', 'url', 'title', 'modified', 'lastmodtime', 'type', 'domain',
       'status', 'part', 'bloc', 'tags', 'image_url', 'pubdate', 'text',
       'rightcol', 'snippet'],
      dtype='object')

In [19]:
df = df[["rightcol"]]

In [26]:
df = df[:1000]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541426 entries, 0 to 541425
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   rightcol  541426 non-null  object
dtypes: object(1)
memory usage: 4.1+ MB


In [ ]:
# Записываем строку в файл
with open('output.txt', 'w') as file:
    for index, row in df.iterrows():
        # Форматируем строку для записи
        line = f"{row['rightcol']}\n"
        file.write(line)

print("Строки DataFrame успешно записаны в отдельные текстовые файлы.")

In [31]:
# # Перебираем строки DataFrame
# for index, row in df.iterrows():
#     # Формируем строку для записи в файл
#     line = ', '.join([str(value) for value in row.values])

#     # Записываем строку в файл
#     with open('output.txt', 'w') as file:
#         for index, row in df.iterrows():
#             # Форматируем строку для записи
#             line = f"{row['rightcol']}\n"
#             file.write(line)

# print("Строки DataFrame успешно записаны в отдельные текстовые файлы.")

KeyboardInterrupt: 

In [ ]:
# Загружаем и создаем датасет
train_path = 'train_dataset.txt'
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Параметры для обучения
training_args = TrainingArguments(
    output_dir="./finetuned",       # директория с выходными данными
    overwrite_output_dir=True,      # перезаписываем содержимое выходной директории при каждом запуске
    num_train_epochs=200,           # число эпох обучения
    per_device_train_batch_size=32, # batch size для обучения
    per_device_eval_batch_size=32,  # batch size для выполнения
    warmup_steps=10,                # количество шагов для "прогрева" (управление скоростью обучения)
    gradient_accumulation_steps=16,) # накопление градиента (16 шагов накапливаем градиенты для batch_size, эмуляция вычисления на пакете 16 * 32 для слабых GPU)

# Инициализируем класс обучения
trainer = Trainer(
    model=model,                 # модель
    args=training_args,          # параметры обучения
    data_collator=data_collator, # загрузчик данных
    train_dataset=train_dataset, # датасет для обучения
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)) # оптимизатор

In [ ]:
# Запуск обучения
trainer.train()

In [ ]:
text = "Почему GPT так жестока?\nМожет просто она - человек?"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
print()

In [ ]:
out = model.generate(input_ids,
                    do_sample=True,
                    num_beams=3,
                    temperature=1.1,
                    top_k=20,
                    top_p=0.8,
                    max_length=70,)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)